<a href="https://colab.research.google.com/github/karthik6717/GenAI/blob/master/Reasoning_LLMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print("Installing necessary libraries...")

# Transformers: Lets you easily use powerful pre-trained AI models for tasks like text generation, translation, or classification.
# Accelerate: Helps you run and train models efficiently across CPUs, GPUs.
# BitsAndBytes: Reduces the size of large models using quantization so they can fit and run on limited hardware like free GPUs.
# Torch (PyTorch): A deep learning library used to build, train, and run neural networks using tensors and GPU acceleration.

# Added 'datasets' to the list
!pip install -q transformers accelerate bitsandbytes torch datasets gradio

print("Libraries installed successfully!")

Installing necessary libraries...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 6.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.9.0+cu126 requires torch==2.9.0, but you have torch 2.9.1 which is incompatible.
torchvision 0.24.0+cu126 requires torch==2.9.0, but you have torch 2.9.1 which is incompatible.
Libraries installed successfully!


In [3]:
# Hugging Face Login

import os
from huggingface_hub import login, notebook_login

print("Attempting Hugging Face login...")

notebook_login()
print("Login successful (or token already present)!")

Attempting Hugging Face login...


Login successful (or token already present)!


In [4]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from datasets import load_dataset  # Import the dataset loading function
import gradio as gr
from IPython.display import display, Markdown
import random  # To pick random news items

# Check for GPU availability
if torch.cuda.is_available():
    print(f"GPU detected: {torch.cuda.get_device_name(0)}")
    torch.set_default_device("cuda")
    print("PyTorch default device set to CUDA (GPU).")
else:
    print("WARNING: No GPU detected. Performance will be very slow.")
    print("Go to Runtime > Change runtime type and select GPU.")

GPU detected: Tesla T4
PyTorch default device set to CUDA (GPU).


In [5]:
# Helper function for markdown display
def print_markdown(text):
    """Displays text as Markdown."""
    display(Markdown(text))

In [6]:
#dataset_id = "mohit9999/all_news_finance_sm_1h2023_custom"
dataset_id = "KrossKinetic/all_news_finance_jsonl"



print(f"Loading dataset: {dataset_id}...")

# Load the dataset (will download if not cached)
# We might only need the 'train' split, specify split = 'train' if needed
# The datatype of news_dataset is datasets.Dataset (from the datasets library by Hugging Face).
news_dataset = load_dataset(dataset_id, split = "train")
print("Dataset loaded successfully!")
#print_markdown(f"**Note:** Changed `dataset_id` from 'PaulAdversarial/all_news_finance_sm_1h2023' to '{dataset_id}' because the original dataset was not found on Hugging Face Hub.")

Loading dataset: KrossKinetic/all_news_finance_jsonl...


README.md:   0%|          | 0.00/214 [00:00<?, ?B/s]

all_news.jsonl: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/5062 [00:00<?, ? examples/s]

Dataset loaded successfully!


In [7]:
# Let's Inspect the dataset
news_dataset

Dataset({
    features: ['_id', 'main_domain', 'title', 'description', 'created_at'],
    num_rows: 5062
})

In [8]:
# Let's display the features (columns and their types)
print("\n Dataset Features")
print(news_dataset.features)


 Dataset Features
{'_id': Value('string'), 'main_domain': Value('string'), 'title': Value('string'), 'description': Value('string'), 'created_at': Value('string')}


In [9]:
# Let's view the datasets as a Pandas DataFrame
news_dataset.to_pandas()

,_id,main_domain,title,description,created_at
0,6453d70d358e80adbfc4cb2b,cnbc.com,"Dow drops 400 points, turns negative for the y...",Regional banks led the broader market lower as...,2023-05-04T16:01:46.448Z
1,6453cf909a78e3af538abe44,cointelegraph.com,Bitcoin drops with stocks as analyst warns of ...,Bitcoin dips as the U.S. banking crisis engulf...,2023-05-04T15:25:28.809Z
2,6453cb87ccab8508100df076,co.uk,Bitcoin Price Analysis: 29370 Tested After Su...,Bitcoin (BTC/USD) sought to add to recent gain...,2023-05-04T15:12:00.971Z
3,6453afd269f3c1643cf0a4f6,bitcoinist.com,"Bitcoin Is 75% To Halving, Here's How Past Cyc...",The current Bitcoin cycle is now 75% on the wa...,2023-05-04T13:10:51.220Z
4,645399d92471d73ea0976d27,seekingalpha.com,"Iron Mountain FFO of $0.71 beats by $0.03, rev...",Iron Mountain press release (IRM): Q1 FFO of...,2023-05-04T11:41:12.498Z
...,...,...,...,...,...
5057,631b16e628ebb650077c97ff,cointelegraph.com,"Ethereum Merge can trigger high volatility, Bi...",BitMEX CEO Alexander Höptner said that their t...,2022-09-09T10:32:52.512Z
5058,6317e6bc2e929627be3d279c,apnews.com,EXPLAINER: Is COVID-19 winding down? Scientist...,Is the coronavirus on its way out? You might t...,2022-09-07T00:30:51.325Z
5059,6317e258adff9198f5a06995,bitcoinmagazine.com,How Bitcoin Educates The World About Finance,Bitcoin brings financial education to the fore...,2022-09-07T00:09:33.167Z
5060,631743e12107b12c267b21dc,coindesk.com,Crypto Hedge Fund Three Arrows Capital Wallet ...,The removal of liquidity was the first activit...,2022-09-06T12:58:08.319Z


In [10]:
# Let's prepare the data for the LLM
# We'll combine title and description for the input text
def combine_news_text(example):

    # Handle potential None values gracefully
    title = example.get("title", "") or ""
    description = example.get("description", "") or ""

    # Add a separator for clarity
    return {"full_text": f"Title: {title}\nDescription: {description}"}

In [11]:
# Let's apply the function to combine the 'title' and 'description' into 'full_text'
# This uses map, which is efficient for datasets

news_dataset = news_dataset.map(combine_news_text)

print("\n--- Sample Data with 'full_text' ---")
print(news_dataset[0])

Map:   0%|          | 0/5062 [00:00<?, ? examples/s]


--- Sample Data with 'full_text' ---
{'_id': '6453d70d358e80adbfc4cb2b', 'main_domain': 'cnbc.com', 'title': 'Dow drops 400 points, turns negative for the year as bank fears grow: Live updates', 'description': 'Regional banks led the broader market lower as contagion fears resurfaced.', 'created_at': '2023-05-04T16:01:46.448Z', 'full_text': 'Title: Dow drops 400 points, turns negative for the year as bank fears grow: Live updates\nDescription: Regional banks led the broader market lower as contagion fears resurfaced.'}


In [12]:
# Let's display the full_text of the first sample
print(news_dataset[0]["full_text"])

Title: Dow drops 400 points, turns negative for the year as bank fears grow: Live updates
Description: Regional banks led the broader market lower as contagion fears resurfaced.


In [13]:
#!pip install --upgrade --force-reinstall bitsandbytes
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# Let's choose our smaller DeepSeek model, which is suitable for Google Colab and reasoning tasks.
model_id = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
print(f"Loading model: {model_id}")
print("This may take a few minutes...")

# AutoModelForCausalLM.from_pretrained(...): Loads a pre-trained language model for tasks like text generation (e.g., ChatGPT-like behavior).
# model_id: The name or path of the model you want to load from Hugging Face (like "meta-llama/Llama-3-8b").
# torch_dtype="auto": Automatically chooses the best data type (like float16 or float32) based on your hardware for efficiency.
# load_in_4bit=True: Loads the model in 4-bit precision to save memory and run on limited hardware like free Colab GPUs.
# device_map="auto": Automatically puts the model on the right device (GPU if available, otherwise CPU).
# trust_remote_code=True: Use only with trusted models to avoid security risks.

model = AutoModelForCausalLM.from_pretrained(model_id,
                                             dtype = "auto",
                                             load_in_4bit = True,
                                             device_map = "auto",
                                             trust_remote_code = True)

model.eval()
print("Model loaded successfully in 4-bit!")

Loading model: deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B
This may take a few minutes...


config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

Model loaded successfully in 4-bit!


In [14]:
# Let's load the Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code = True)
print("Tokenizer loaded successfully.")

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Tokenizer loaded successfully.


In [15]:
# Let's create the Text Generation Pipeline
llm_pipeline = pipeline("text-generation", model = model, tokenizer = tokenizer, torch_dtype = "auto", device_map = "auto")
print("Text generation pipeline created successfully.")

`torch_dtype` is deprecated! Use `dtype` instead!
Device set to use cuda:0


Text generation pipeline created successfully.


In [16]:
# Let's build the full formatted prompt:
# <|im_start|>user: Marks the beginning of the user's message.
# {test_question}: Inserts your question.
# <|im_end|>: Marks the end of the user’s message.
# <|im_start|>assistant: Signals that the assistant (AI) is about to reply.
# <think>: Encourages the AI to generate internal reasoning or thoughts before answering.

test_question = "Explain what electric cars are in simple terms. Keep the thinking short."
test_prompt = f"<|im_start|>user\n{test_question}<|im_end|>\n<|im_start|>assistant\n<think>\n"
print(f"Test prompt:\n{test_prompt}")

Test prompt:
<|im_start|>user
Explain what electric cars are in simple terms. Keep the thinking short.<|im_end|>
<|im_start|>assistant
<think>



In [17]:
# Let's test this beast of a model!
# Generate text using the pipeline
# max_new_tokens: Maximum number of tokens to generate
# do_sample: Enable sampling for more diverse outputs (vs greedy decoding). Enables sampling, meaning the model won’t always pick the highest probability next word — instead, it adds some randomness to make responses more creative and diverse.
# temperature: Controls randomness (0.7 is balanced between creative and focused)
# top_p: Nucleus sampling parameter (0.9 keeps responses on topic while allowing variety). Where the model chooses from the top 90% of likely next words (not just the top one). Helps keep output fluent but varied.

print("Testing model with a simple instruction")

outputs = llm_pipeline(test_prompt,
                       max_new_tokens = 4000,
                       do_sample = True,
                       temperature = 0.7,
                       top_p = 0.9)

outputs

Testing model with a simple instruction


[{'generated_text': "<|im_start|>user\nExplain what electric cars are in simple terms. Keep the thinking short.<|im_end|>\n<|im_start|>assistant\n<think>\nOkay, so I'm trying to explain what electric cars are in simple terms. Let me break it down. Electric cars are vehicles that use electricity to power them, right? Unlike regular cars that use gas, these cars generate energy internally. That makes sense because using electricity is more efficient and environmentally friendly.\n\nI should mention that they can start moving without any gas, which is a big plus. Also, since they don't need fuel, they can be used for short trips or even for covering a distance if they have an external source of fuel. This makes the cars more versatile and eco-friendly.\n\nI should keep the explanation short and in simple terms, avoiding any technical jargon. Maybe use examples or comparisons to make it clearer. Let me try to put it all together in a concise and easy-to-understand manner.\n</think>\n\nElec

In [18]:
# Function to extract and format the reason and output
def format_model_output(output):

    # Extract the content within <think> tags as Reason
    reason_start = output.find("<think>") + len("<think>")
    reason_end = output.find("</think>")
    reason = output[reason_start:reason_end].strip()

    # Extract the content after </think> as Output
    output_start = reason_end + len("</think>")
    model_output_content = output[output_start:].strip()

    # Format the result
    reason = f"**Reason**:\n{reason}\n"
    output = f"**Output**:\n{model_output_content}"
    return reason, output

In [19]:
# Extract and print the generated text (only the response part)
full_output = outputs[0]["generated_text"]
reason, output = format_model_output(full_output)
print_markdown(reason + output)

**Reason**:
Okay, so I'm trying to explain what electric cars are in simple terms. Let me break it down. Electric cars are vehicles that use electricity to power them, right? Unlike regular cars that use gas, these cars generate energy internally. That makes sense because using electricity is more efficient and environmentally friendly.

I should mention that they can start moving without any gas, which is a big plus. Also, since they don't need fuel, they can be used for short trips or even for covering a distance if they have an external source of fuel. This makes the cars more versatile and eco-friendly.

I should keep the explanation short and in simple terms, avoiding any technical jargon. Maybe use examples or comparisons to make it clearer. Let me try to put it all together in a concise and easy-to-understand manner.
**Output**:
Electric cars are vehicles powered by electricity instead of gas. They start moving without needing fuel and can be used for short trips or even with an external source of fuel. This makes them efficient and environmentally friendly.

In [20]:
import re  # Import regular expressions for parsing

def analyze_news_sentiment(news_text, llm_pipeline):
    """
    Analyzes news sentiment using the LLM, providing reasoning and classification.

    Args:
        news_text (str): The combined title and description of the news.
        llm_pipeline (transformers.pipeline): The initialized text-generation pipeline.

    Returns:
        str: A string containing the reasoning and classification.
    """

    # Define the Prompt Template with specific instructions for the model
    test_question = f"""You are a concise Financial News Analyst.
    Analyze the provided news text. Output ONLY the three requested items below, each on a new line, prefixed with the specified label.

    1.  REASONING: Brief financial reasoning (1-2 points max) for the sentiment.
    2.  SENTIMENT: Choose ONE: Positive, Negative, Neutral.
    3.  TAG: A concise topic tag (1-3 words).

    Do NOT add any other text, greetings, or explanations.

    News Text:
    {news_text}"""

    # Format the prompt according to DeepSeek's expected input format with thinking tags
    test_prompt = f"<|im_start|>user\n{test_question}<|im_end|>\n<|im_start|>assistant\n<think>\n"

    # Print the prompt for debugging purposes to verify what's being sent to the model
    print(f"Test prompt:\n{test_prompt}")

    # Run the model inference with specific generation parameters
    outputs = llm_pipeline(
        test_prompt,
        max_new_tokens = 4000,
        do_sample = True,
        temperature = 0.7,
        top_p = 0.9)

    # Extract the full generated text and parse it to separate reasoning from classification
    # The format_model_output function likely separates the thinking process from the final answer
    full_output = outputs[0]["generated_text"]
    reason, output = format_model_output(full_output)

    # Return both the reasoning process and the final sentiment classification
    return reason, output


In [21]:

# Print a separator line for clarity in the output
print("\n" + "=" * 30 + " TESTING ANALYSIS " + "=" * 30)


# Select a few random indices from the news dataset to test the analysis function
random_indices = random.sample(range(len(news_dataset)), 3)

# Iterate over each randomly selected index
for index in random_indices:
    # Retrieve the full text of the news item at the current index
    sample_news = news_dataset[index]["full_text"]

    # Analyze the sentiment of the sample news using the sentiment analysis function
    reason, output = analyze_news_sentiment(sample_news, llm_pipeline)

    # Print the analysis result header for the current index
    print(f"\n--- Analysis Result for Index {index} ---")

    # Display the reasoning and output in a formatted markdown style
    print_markdown(f"{reason}\n\n{output}")

    # Print a separator line for better readability between results
    print("-" * 60)




============================== TESTING ANALYSIS ==============================
Test prompt:
<|im_start|>user
You are a concise Financial News Analyst.
    Analyze the provided news text. Output ONLY the three requested items below, each on a new line, prefixed with the specified label.

    1.  REASONING: Brief financial reasoning (1-2 points max) for the sentiment.
    2.  SENTIMENT: Choose ONE: Positive, Negative, Neutral.
    3.  TAG: A concise topic tag (1-3 words).

    Do NOT add any other text, greetings, or explanations.

    News Text:
    Title: Law of One Price: Definition, Example, Assumptions
Description: The law of one price is the theory that an economic good or asset will have the same price in different markets, given certain assumptions.<|im_end|>
<|im_start|>assistant
<think>


--- Analysis Result for Index 4408 ---


**Reason**:
Okay, so I'm trying to figure out the three items for the given news text about the Law of One Price. Let me break it down step by step.

First, I need to understand what the Law of One Price is. From the description, it's a theory that different markets should have the same price for an economic good or asset, provided certain assumptions are met. So, the reasoning here would be that if all markets have the same price, it's easier for investors to make decisions, which is a fundamental concept in financial markets.

Next, determining the sentiment. The Law of One Price is a foundational concept, so I think the sentiment is likely Neutral. It's not a strong positive or negative statement; it's more about an established principle.

Lastly, the tag. It should be concise, so "Law of One Price" is perfect. I don't think I need to add more words or mention other related concepts here.

I need to make sure I don't add any extra text or explanations, just the three items as specified. Let me double-check that I'm only including the reasoning, sentiment, and tag without any markdown or additional text.


**Output**:
1. REASONING: The Law of One Price is a foundational concept in financial markets, stating that similar economic goods or assets should have the same price across different markets, given certain assumptions.

2. SENTIMENT: Neutral

3. TAG: Law of One Price

------------------------------------------------------------
Test prompt:
<|im_start|>user
You are a concise Financial News Analyst.
    Analyze the provided news text. Output ONLY the three requested items below, each on a new line, prefixed with the specified label.

    1.  REASONING: Brief financial reasoning (1-2 points max) for the sentiment.
    2.  SENTIMENT: Choose ONE: Positive, Negative, Neutral.
    3.  TAG: A concise topic tag (1-3 words).

    Do NOT add any other text, greetings, or explanations.

    News Text:
    Title: 2-year Treasury yield rises after Fed raises rates by half a point, indicates more hikes
Description: The Federal Reserve delivered a 50 basis point rate hike and indicated that it will continue raising rates to tame inflation.<|im_end|>
<|im_start|>assistant
<think>


--- Analysis Result for Index 3782 ---


**Reason**:
Alright, I'm looking at this news text about the 2-year Treasury yield rising after the Federal Reserve raised rates by half a point. The Fed will continue raising rates to manage inflation. So, first, I need to determine the sentiment. The yield is rising, which typically indicates a negative impact on the economy, so it's a negative sentiment.

Next, I have to tag the information. The topic here is the impact of Fed rate hikes on the economy, so the tag should be "Economic Impact" with a single word.

For the reasoning part, I need to explain why the sentiment is negative. The rising yield suggests that the Federal Reserve is trying to control inflation, which might lead to higher interest rates and potentially causing economic instability. This makes the sentiment negative.

I should make sure each item is on a new line and uses the specified labels. No extra text or explanations, just concise information as requested.


**Output**:
1. REASONING: The rising 2-year Treasury yield indicates that the Federal Reserve is raising rates to control inflation, which may lead to economic instability. This is a negative impact on the economy.

2. SENTIMENT: Negative

3. TAG: Economic Impact

------------------------------------------------------------
Test prompt:
<|im_start|>user
You are a concise Financial News Analyst.
    Analyze the provided news text. Output ONLY the three requested items below, each on a new line, prefixed with the specified label.

    1.  REASONING: Brief financial reasoning (1-2 points max) for the sentiment.
    2.  SENTIMENT: Choose ONE: Positive, Negative, Neutral.
    3.  TAG: A concise topic tag (1-3 words).

    Do NOT add any other text, greetings, or explanations.

    News Text:
    Title: UBS eyeing swoop for Credit Suisse, sources say, amid fears of banking contagion
Description: UBS AG was examining on Saturday a takeover of its embattled Swiss peer Credit Suisse , sources said, a move that could dampen fears the unfolding crisis might destabilise the global banking system.<|im_end|>
<|im_start|>assistant
<think>


--- Analysis Result for Index 1057 ---


**Reason**:
Alright, so I'm trying to figure out how to respond to this user's request. They've given me a news text about UBS eyeing a takeover for Credit Suisse, and they want me to analyze it into three specific items: Reasoning, Sentiment, and Tag. Each item should be a single line, and I can't add any extra text or explanations.

First, I need to parse the news text carefully. The main points are that UBS is considering a takeover for Credit Suisse, and it's said that this move could dampen fears of a banking crisis. The sources mention that such a move could affect the global banking system, which is something I should consider.

Now, for the Reasoning part, I should briefly explain why the decision is being made. It's about UBS's focus on stability, so I can say that UBS is looking at a takeover to protect against potential banking crises. That makes sense because they're more concerned about the stability of their own banking operations rather than the broader crisis.

Next, the Sentiment. The user wants me to choose between Positive, Negative, or Neutral. Given that UBS is looking to stabilize their own banking operations, I think the sentiment here is Neutral because it's not a strong positive or negative opinion, just a neutral stance.

Finally, the Tag. This should be a concise topic tag. The key here is the takeoff for UBS, so I'll choose "UBS" as the tag, which is short and covers the main focus of the analysis.

I need to make sure I'm only outputting three items each on a new line. I'll avoid any additional text or explanations and just stick to the structure the user provided. Also, I should keep each item concise, fitting into one line without any markdown or extra formatting.

So, putting it all together, the Reasoning will explain why the move is being considered, the Sentiment will be neutral because it's not a strong opinion, and the Tag will be "UBS" to capture the main focus of the analysis.


**Output**:
1. REASONING: UBS AG is considering a takeover for its embattled peer Credit Suisse, which could help stabilize UBS's own banking operations, potentially addressing the risks of a global banking crisis.

2. SENTIMENT: Neutral

3. TAG: UBS

------------------------------------------------------------


In [1]:
import gradio as gr  # Importing the Gradio library for creating the web interface
import random  # Importing the random library to generate random numbers
import re  # Importing the regular expressions library (not used in this snippet)
from transformers import pipeline  # Importing the pipeline function from transformers for model loading

# --- Gradio Helper Functions ---
def get_random_news():
    """Fetches and returns the full_text of a random news item."""
    if not news_dataset:  # Check if the news dataset is empty
        return "Error: No news items available."  # Return an error message if no news items are found
    random_index = random.randint(0, len(news_dataset) - 1)  # Generate a random index to select a news item
    news_text = news_dataset[random_index]['full_text']  # Retrieve the full text of the news item at the random index
    print(f"Fetched news item at index {random_index}")  # Print the index of the fetched news item
    return news_text  # Return the fetched news text

def perform_analysis(news_text):
    """Triggers analysis on the provided news text."""
    if not news_text or news_text.startswith("Error"):  # Check if the news text is empty or an error message
        return "Error: No news text to analyze."  # Return an error message if no valid news text is provided
    print(f"Analyzing news: {news_text[:50]}...")  # Print the first 50 characters of the news text being analyzed
    reason, output = analyze_news_sentiment(news_text, llm_pipeline)  # Analyze the sentiment of the news text
    return reason, output  # Return the reasoning and output from the analysis

# --- Build Gradio Interface ---
with gr.Blocks(theme=gr.themes.Glass()) as demo:  # Create a Gradio Blocks interface with a glass theme
    gr.Markdown("""
    # DeepSeek Financial News Analyzer
    Fetches a random news item from the dataset.
    Click 'Analyze News' to get sentiment classification and the model's reasoning.

    """)

    with gr.Row():  # Create a row for buttons
        btn_fetch = gr.Button("🔄 Fetch Random News Item")  # Button to fetch a random news item
        btn_analyze = gr.Button("💡 Analyze News", variant="primary")  # Button to analyze the news

    news_display = gr.Textbox(  # Textbox to display the news item
        label="📰 News Text (Title & Description)",  # Label for the textbox
        lines=8,  # Number of lines in the textbox
        interactive=False,  # Make the textbox non-interactive
        placeholder="Click 'Fetch Random News Item' to display news."  # Placeholder text
    )
    # Creates a collapsible panel
    with gr.Accordion("🤖 Model Reason", open=True):  # Accordion for model reasoning
        analysis_display = gr.Markdown()  # Markdown display for the reasoning

    with gr.Accordion("🤖 Model Output", open=True):  # Accordion for model output
        analysis_output = gr.Markdown()  # Markdown display for the output

    # --- Event Handlers ---
    btn_fetch.click(  # Set up click event for the fetch button
        fn=get_random_news,  # Function to call when the button is clicked
        inputs=[],  # No inputs needed
        outputs=[news_display]  # Output to the news display textbox
    )

    btn_analyze.click(  # Set up click event for the analyze button
        fn=perform_analysis,  # Function to call when the button is clicked
        inputs=[news_display],  # Input from the news display textbox
        outputs=[analysis_display, analysis_output]  # Outputs to the reasoning and output displays
    )

    # Load initial news item when the app starts
    demo.load(  # Load function to run when the app starts
        fn=get_random_news,  # Function to call to get a random news item
        inputs=None,  # No inputs needed
        outputs=[news_display]  # Output to the news display textbox
    )

# --- Launch the Gradio App ---
print("Launching Gradio demo...")  # Print message indicating the app is launching
demo.launch(debug=True)  # Launch the Gradio app in debug mode

/tmp/ipython-input-1399811874.py:25: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme=gr.themes.Glass()) as demo:  # Create a Gradio Blocks interface with a glass theme


Launching Gradio demo...
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://026ca8830ddacecf2b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/gradio/queueing.py", line 759, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/gradio/route_utils.py", line 354, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/gradio/blocks.py", line 2191, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/gradio/blocks.py", line 1698, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/anyio/to_thread.py", line 61, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
           ^^^^^

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://026ca8830ddacecf2b.gradio.live
